This notebook documents how to solve the famous [zebra puzzle](http://en.wikipedia.org/wiki/Zebra_puzzle) by A. Einstein with the help of semantic web tools for python.


It is based on theses Links/Projects

- https://pythonhosted.org/Owlready2/index.html
- https://github.com/RDFLib/OWL-RL/issues/3 "Solving Einstein's riddle (zebra puzzle"
    - Linked owl-files:
    - https://github.com/RDFLib/OWL-RL/files/1533408/einsteins_riddle.owl.txt ← works
    - https://github.com/RDFLib/OWL-RL/files/1533409/zebra.n3.txt  ← works not yet
- http://lambdamusic.github.io/Ontospy/


For installation I did:

```bash
pip install owlready
pip install ontospy[FULL]
``` 


In [1]:
import os
import owlready2 as owl2

# Direct approach

we assume that both ontology files have been downloaded to `./ontology_data/<filename>`. They are parsed directly with owlready.

## Ontology 1

In [2]:
data_path = "ontology_data"
path1 = os.path.join(data_path, "einsteins_riddle.owl.txt")


In [3]:
onto = owl2.get_ontology(path1).load()
print("classes (concepts):\n", list(onto.classes()), "\n"*2)
print("properties (roles):\n", list(onto.properties()), "\n"*2)
print("individuals:\n", list(onto.individuals()))

classes (concepts):
 [owl.Nothing, einsteins_riddle.owl.txt.Beverage, einsteins_riddle.owl.txt.Man, einsteins_riddle.owl.txt.Color, einsteins_riddle.owl.txt.House, einsteins_riddle.owl.txt.Pet, einsteins_riddle.owl.txt.Cigarette] 


properties (roles):
 [einsteins_riddle.owl.txt.drinks, einsteins_riddle.owl.txt.has_color, einsteins_riddle.owl.txt.lives_in, einsteins_riddle.owl.txt.owns, einsteins_riddle.owl.txt.right_to, einsteins_riddle.owl.txt.smokes, 1.1.description, 1.1.date, 1.1.creator] 


individuals:
 [einsteins_riddle.owl.txt.orange_juice, einsteins_riddle.owl.txt.tea, einsteins_riddle.owl.txt.water, einsteins_riddle.owl.txt.coffee, einsteins_riddle.owl.txt.milk, einsteins_riddle.owl.txt.Kools, einsteins_riddle.owl.txt.Lucky_Strike, einsteins_riddle.owl.txt.Parliaments, einsteins_riddle.owl.txt.Chesterfields, einsteins_riddle.owl.txt.Old_Gold, einsteins_riddle.owl.txt.green, einsteins_riddle.owl.txt.red, einsteins_riddle.owl.txt.blue, einsteins_riddle.owl.txt.yellow, einsteins

### Relations before calling the reasoner

In [4]:
list(onto.lives_in.get_relations())

[(einsteins_riddle.owl.txt.Norwegian, einsteins_riddle.owl.txt.house_1)]

In [5]:
list(onto.owns.get_relations())

[(einsteins_riddle.owl.txt.Spaniard, einsteins_riddle.owl.txt.dog)]

### Calling the reasoner and inspect relations again

In [6]:
%time owl2.sync_reasoner_pellet(infer_property_values=True, infer_data_property_values=True, debug=0)

CPU times: user 15 ms, sys: 3.82 ms, total: 18.9 ms
Wall time: 3.93 s


In [7]:
rels = list(onto.lives_in.get_relations())
print(rels)

[(einsteins_riddle.owl.txt.Norwegian, einsteins_riddle.owl.txt.house_1), (einsteins_riddle.owl.txt.Englishman, einsteins_riddle.owl.txt.house_3), (einsteins_riddle.owl.txt.Japanese, einsteins_riddle.owl.txt.house_5), (einsteins_riddle.owl.txt.Ukrainian, einsteins_riddle.owl.txt.house_2), (einsteins_riddle.owl.txt.Spaniard, einsteins_riddle.owl.txt.house_4)]


In [8]:
rels = list(onto.owns.get_relations())
print(rels)

[(einsteins_riddle.owl.txt.Spaniard, einsteins_riddle.owl.txt.dog), (einsteins_riddle.owl.txt.Englishman, einsteins_riddle.owl.txt.snails), (einsteins_riddle.owl.txt.Japanese, einsteins_riddle.owl.txt.zebra), (einsteins_riddle.owl.txt.Norwegian, einsteins_riddle.owl.txt.fox), (einsteins_riddle.owl.txt.Ukrainian, einsteins_riddle.owl.txt.horse)]


In [9]:
for r in rels:
    print(f"assert n.{r[0].name}.lives_in == n.{r[1].name}")


assert n.Spaniard.lives_in == n.dog
assert n.Englishman.lives_in == n.snails
assert n.Japanese.lives_in == n.zebra
assert n.Norwegian.lives_in == n.fox
assert n.Ukrainian.lives_in == n.horse


### Conclusion

The puzzle could be solved easily by the reasoner